In [6]:
# Cella 1: Import librerie
import numpy as np
from IPython.display import display, Markdown

In [7]:
# Cella 2: Caricamento dati dalla scheda 'DATI CICLI'
# Dalla tabella 'DATI CICLI' nel file Excel

# Dati per ogni ciclo (ciclo 1 a ciclo 6)
# Colonna A: Minimi, Colonna B: Massimi, Colonna D: Massimo, Colonna E: Minimo, Colonna F: Massimo
# Calcoliamo anche Durata PI/2 (colonne G e H)

dati_cicli = np.array([
    [45, 15, 15, 45, 116],  # Ciclo 1
    [156, 116, 116, 156, 217],  # Ciclo 2
    [269, 217, 217, 269, 312],  # Ciclo 3
    [374, 312, 312, 374, 404],  # Ciclo 4
    [439, 404, 404, 439, 494],  # Ciclo 5
    [535, 494, 494, 535, 597]   # Ciclo 6
])

# Calcolo Durata PI/2 come nel file Excel
# Colonna G: =E-D, Colonna H: =F-E
durata_pi2_1 = dati_cicli[:, 3] - dati_cicli[:, 2]  # G: E-D
durata_pi2_2 = dati_cicli[:, 4] - dati_cicli[:, 3]  # H: F-E

# Aggiungiamo le durate ai dati
dati_cicli_completi = np.column_stack((dati_cicli, durata_pi2_1, durata_pi2_2))

print("DATI CICLI completi (Minimi, Massimi, D, E, F, G, H):")
print(dati_cicli_completi)
print("\nDurata PI/2 (G):", durata_pi2_1)
print("Durata PI/2 (H):", durata_pi2_2)

DATI CICLI completi (Minimi, Massimi, D, E, F, G, H):
[[ 45  15  15  45 116  30  71]
 [156 116 116 156 217  40  61]
 [269 217 217 269 312  52  43]
 [374 312 312 374 404  62  30]
 [439 404 404 439 494  35  55]
 [535 494 494 535 597  41  62]]

Durata PI/2 (G): [30 40 52 62 35 41]
Durata PI/2 (H): [71 61 43 30 55 62]


In [8]:
# Cella 3: Preparazione dati per la scheda 'CALCOLO CICLI'
# Analizziamo la struttura delle formule nel file Excel:

# Per le prime 31 righe (eccetto riga 31), la colonna D usa: =A/'DATI CICLI'!G$2+1
# Dove G$2 = durata_pi2_1[0] (valore fisso per ciclo 1)

# Per la riga 31: ='DATI CICLI'!A2/('DATI CICLI'!G$2+1)
# A2 è 15 (il minimo del ciclo 1)

# Dalla riga 71 alla 103: =A/'DATI CICLI'!H$2
# Dove H$2 = durata_pi2_2[0] (valore fisso per ciclo 1)

# Dalla riga 104 alla 256: pattern simile ma cambia riferimento a G$3, H$3, etc.
# per ogni ciclo successivo

print("Dati per i calcoli:")
print(f"Durata G per ciclo 1 (G$2): {durata_pi2_1[0]}")
print(f"Durata H per ciclo 1 (H$2): {durata_pi2_2[0]}")
print(f"Valore A2 (minimo ciclo 1): {dati_cicli[0, 1]}")  # Attenzione: nella tabella B2 è 15

Dati per i calcoli:
Durata G per ciclo 1 (G$2): 30
Durata H per ciclo 1 (H$2): 71
Valore A2 (minimo ciclo 1): 15


In [9]:
# Cella 4: Ricostruzione della logica di calcolo
# La colonna A in 'CALCOLO CICLI' sembra essere un contatore che varia
# La colonna B sembra essere "Migliaia di anni" con valori crescenti e poi decrescenti
# La colonna C è "Segnale Vulcanico" (dati forniti)

# Ricostruiamo la struttura del calcolo
def calcola_valore_ciclo(indice_riga, ciclo_corrente=1, fase="G"):
    """
    Simula il calcolo della colonna D in 'CALCOLO CICLI'
    
    Parametri:
    - indice_riga: indice corrente (0-based)
    - ciclo_corrente: quale ciclo stiamo processando (1-6)
    - fase: "G" per prima metà ciclo, "H" per seconda metà
    """
    ciclo_idx = ciclo_corrente - 1  # Converti a 0-based
    
    if fase == "G":
        # =A/'DATI CICLI'!G$n+1 (dove n = ciclo_corrente+1)
        # Nel file: G$2 per ciclo 1, G$3 per ciclo 2, etc.
        denominatore = dati_cicli_completi[ciclo_idx, 5] + 1  # G + 1
    else:  # fase == "H"
        # =A/'DATI CICLI'!H$n
        denominatore = dati_cicli_completi[ciclo_idx, 6]  # H
    
    # Per ora restituiamo solo il denominatore, il valore A verrà applicato dopo
    return denominatore

# Test per i primi cicli
print("Denominatori per i calcoli:")
print(f"Ciclo 1, fase G: {dati_cicli_completi[0, 5] + 1}")
print(f"Ciclo 1, fase H: {dati_cicli_completi[0, 6]}")
print(f"Ciclo 2, fase G: {dati_cicli_completi[1, 5] + 1}")
print(f"Ciclo 2, fase H: {dati_cicli_completi[1, 6]}")

Denominatori per i calcoli:
Ciclo 1, fase G: 31
Ciclo 1, fase H: 71
Ciclo 2, fase G: 41
Ciclo 2, fase H: 61


In [10]:
# Cella 5: Creazione struttura dati per 'CALCOLO CICLI'
# Basato sull'analisi del file, ci sono blocchi di dati per ogni ciclo

# Definiamo i blocchi per ogni ciclo
# Ogni ciclo ha:
# 1. Fase G: un certo numero di righe
# 2. Fase H: un certo numero di righe

# Dal file Excel, contiamo approssimativamente:
blocchi_cicli = [
    {"fase_G_righe": 30, "fase_H_righe": 33},  # Ciclo 1: righe 1-30 (G), 71-103 (H)
    {"fase_G_righe": 30, "fase_H_righe": 30},  # Ciclo 2: righe 104-133 (G), 157-186 (H)
    {"fase_G_righe": 30, "fase_H_righe": 30},  # Ciclo 3: righe 187-216 (G), 242-271 (H)
    {"fase_G_righe": 30, "fase_H_righe": 30},  # Ciclo 4: righe 272-301 (G), 327-356 (H)
    {"fase_G_righe": 30, "fase_H_righe": 30},  # Ciclo 5: righe 357-386 (G), 392-421 (H)
    {"fase_G_righe": 30, "fase_H_righe": 30},  # Ciclo 6: righe 422-451 (G), 477-506 (H)
]

# Il numero di righe nel file Excel è molto maggiore, ma questo è lo schema
totale_righe = sum([b["fase_G_righe"] + b["fase_H_righe"] for b in blocchi_cicli])
print(f"Totale righe approssimative nei blocchi ciclo: {totale_righe}")
print(f"Righe effettive nel file: 481 (dalla tabella fornita)")

Totale righe approssimative nei blocchi ciclo: 363
Righe effettive nel file: 481 (dalla tabella fornita)


In [11]:
# Cella 6: Creazione di array per 'CALCOLO CICLI' basato sui dati forniti
# Dalla tabella, estraiamo manualmente i dati chiave:

# Colonna A: "Riga" - valori da 1 a 62, poi da 71 a 1, poi pattern ripetuti
# Colonna B: "Migliaia di anni" - da 1 a 117, poi da 46 a 116, etc.
# Colonna C: "Segnale Vulcanico" - dati forniti (0, 1, 2, 3, 4)
# Colonna D: "Valore ciclo" - calcolato come A/denominatore
# Colonna E: "1 - valore ciclo" - calcolato come 1 - D
# Colonna F: "Valore negativo" - calcolato come -E o +E a seconda del segno

# Dato che non abbiamo tutti i valori esatti della colonna A e B,
# implementiamo la logica di calcolo:

def calcola_riga_ciclo(valore_A, denominatore, segno_negativo=True):
    """
    Calcola i valori per una riga della scheda 'CALCOLO CICLI'
    
    Parametri:
    - valore_A: valore della colonna A per questa riga
    - denominatore: valore da usare nel calcolo (da DATI CICLI)
    - segno_negativo: True per -E, False per +E (come nel file)
    """
    # Colonna D: =A/denominatore
    D = valore_A / denominatore
    
    # Colonna E: =1-D
    E = 1 - D
    
    # Colonna F: =-E o =+E a seconda del segno
    if segno_negativo:
        F = -E
    else:
        F = E
    
    return D, E, F

# Test con alcuni valori
print("Test calcoli:")
test_denominatori = [dati_cicli_completi[0, 5] + 1, dati_cicli_completi[0, 6]]

for i, denom in enumerate(test_denominatori):
    D, E, F = calcola_riga_ciclo(valore_A=10, denominatore=denom, segno_negativo=(i==0))
    print(f"Denominatore {denom:.1f}, A=10: D={D:.4f}, E={E:.4f}, F={F:.4f}")

Test calcoli:
Denominatore 31.0, A=10: D=0.3226, E=0.6774, F=-0.6774
Denominatore 71.0, A=10: D=0.1408, E=0.8592, F=0.8592


In [12]:
# Cella 7: Ricostruzione del processo completo
# Simuliamo il calcolo per tutti i 6 cicli

risultati_completi = []

for ciclo_idx in range(6):  # Cicli da 0 a 5 (ciclo 1 a 6)
    ciclo_num = ciclo_idx + 1
    
    # Fase G del ciclo (prima metà)
    denominatore_G = dati_cicli_completi[ciclo_idx, 5] + 1
    
    # Fase H del ciclo (seconda metà)
    denominatore_H = dati_cicli_completi[ciclo_idx, 6]
    
    print(f"\n{'='*50}")
    print(f"CICLO {ciclo_num}")
    print(f"{'='*50}")
    print(f"Denominatore fase G: {denominatore_G}")
    print(f"Denominatore fase H: {denominatore_H}")
    
    # Calcolo per la fase G (usando segno negativo per F)
    print("\nFase G (esempio primi 5 valori):")
    for i in range(1, 6):  # Primi 5 valori come esempio
        D, E, F = calcola_riga_ciclo(i, denominatore_G, segno_negativo=True)
        print(f"  A={i}: D={D:.4f}, E={E:.4f}, F={F:.4f}")
        risultati_completi.append({
            'ciclo': ciclo_num,
            'fase': 'G',
            'A': i,
            'D': D,
            'E': E,
            'F': F
        })
    
    # Calcolo per la fase H (usando segno positivo per F)
    print("\nFase H (esempio primi 5 valori):")
    for i in range(1, 6):  # Primi 5 valori come esempio
        D, E, F = calcola_riga_ciclo(i, denominatore_H, segno_negativo=False)
        print(f"  A={i}: D={D:.4f}, E={E:.4f}, F={F:.4f}")
        risultati_completi.append({
            'ciclo': ciclo_num,
            'fase': 'H',
            'A': i,
            'D': D,
            'E': E,
            'F': F
        })


CICLO 1
Denominatore fase G: 31
Denominatore fase H: 71

Fase G (esempio primi 5 valori):
  A=1: D=0.0323, E=0.9677, F=-0.9677
  A=2: D=0.0645, E=0.9355, F=-0.9355
  A=3: D=0.0968, E=0.9032, F=-0.9032
  A=4: D=0.1290, E=0.8710, F=-0.8710
  A=5: D=0.1613, E=0.8387, F=-0.8387

Fase H (esempio primi 5 valori):
  A=1: D=0.0141, E=0.9859, F=0.9859
  A=2: D=0.0282, E=0.9718, F=0.9718
  A=3: D=0.0423, E=0.9577, F=0.9577
  A=4: D=0.0563, E=0.9437, F=0.9437
  A=5: D=0.0704, E=0.9296, F=0.9296

CICLO 2
Denominatore fase G: 41
Denominatore fase H: 61

Fase G (esempio primi 5 valori):
  A=1: D=0.0244, E=0.9756, F=-0.9756
  A=2: D=0.0488, E=0.9512, F=-0.9512
  A=3: D=0.0732, E=0.9268, F=-0.9268
  A=4: D=0.0976, E=0.9024, F=-0.9024
  A=5: D=0.1220, E=0.8780, F=-0.8780

Fase H (esempio primi 5 valori):
  A=1: D=0.0164, E=0.9836, F=0.9836
  A=2: D=0.0328, E=0.9672, F=0.9672
  A=3: D=0.0492, E=0.9508, F=0.9508
  A=4: D=0.0656, E=0.9344, F=0.9344
  A=5: D=0.0820, E=0.9180, F=0.9180

CICLO 3
Denominator

In [13]:
# Cella 8: Analisi della colonna A (valori di riga)
# Notiamo dal file che i valori nella colonna A seguono pattern specifici:
# - Per la fase G: valori crescenti (1, 2, 3, ...)
# - Per la fase H: valori decrescenti (71, 70, 69, ... per ciclo 1)

# Ricostruiamo il pattern completo basato sui minimi e massimi
print("Pattern dei valori nella colonna A per ogni ciclo:")

for ciclo_idx in range(6):
    ciclo_num = ciclo_idx + 1
    
    # Dalla scheda DATI CICLI:
    # Per fase G: valori da 1 a (E-D) = durata_pi2_1
    # Per fase H: valori da (F-E) = durata_pi2_2 a 1
    
    max_G = int(dati_cicli_completi[ciclo_idx, 5])  # durata_pi2_1
    max_H = int(dati_cicli_completi[ciclo_idx, 6])  # durata_pi2_2
    
    print(f"\nCiclo {ciclo_num}:")
    print(f"  Fase G: valori A da 1 a {max_G}")
    print(f"  Fase H: valori A da {max_H} a 1")

Pattern dei valori nella colonna A per ogni ciclo:

Ciclo 1:
  Fase G: valori A da 1 a 30
  Fase H: valori A da 71 a 1

Ciclo 2:
  Fase G: valori A da 1 a 40
  Fase H: valori A da 61 a 1

Ciclo 3:
  Fase G: valori A da 1 a 52
  Fase H: valori A da 43 a 1

Ciclo 4:
  Fase G: valori A da 1 a 62
  Fase H: valori A da 30 a 1

Ciclo 5:
  Fase G: valori A da 1 a 35
  Fase H: valori A da 55 a 1

Ciclo 6:
  Fase G: valori A da 1 a 41
  Fase H: valori A da 62 a 1


In [14]:
# Cella 9: Funzione per generare tutti i calcoli completi

def genera_calcoli_completi():
    """
    Genera tutti i calcoli come nel file Excel per tutti i 6 cicli
    """
    tutti_calcoli = []
    
    for ciclo_idx in range(6):
        ciclo_num = ciclo_idx + 1
        
        # Valori dalla scheda DATI CICLI
        durata_G = dati_cicli_completi[ciclo_idx, 5]  # Colonna G
        durata_H = dati_cicli_completi[ciclo_idx, 6]  # Colonna H
        
        denominatore_G = durata_G + 1
        denominatore_H = durata_H
        
        # Fase G: valori crescenti, F negativo
        print(f"\nCiclo {ciclo_num} - Fase G ({int(durata_G)} righe):")
        for i in range(1, int(durata_G) + 1):
            D, E, F = calcola_riga_ciclo(i, denominatore_G, segno_negativo=True)
            tutti_calcoli.append({
                'ciclo': ciclo_num,
                'fase': 'G',
                'riga': i,
                'A': i,
                'D': D,
                'E': E,
                'F': F,
                'denominatore': denominatore_G
            })
        
        # Fase H: valori decrescenti, F positivo
        print(f"\nCiclo {ciclo_num} - Fase H ({int(durata_H)} righe):")
        for i in range(int(durata_H), 0, -1):  # Da durata_H a 1
            D, E, F = calcola_riga_ciclo(i, denominatore_H, segno_negativo=False)
            tutti_calcoli.append({
                'ciclo': ciclo_num,
                'fase': 'H',
                'riga': int(durata_H) - i + 1,  # Contatore progressivo
                'A': i,
                'D': D,
                'E': E,
                'F': F,
                'denominatore': denominatore_H
            })
    
    return tutti_calcoli

# Genera tutti i calcoli
calcoli_completi = genera_calcoli_completi()
print(f"\nTotale righe calcolate: {len(calcoli_completi)}")


Ciclo 1 - Fase G (30 righe):

Ciclo 1 - Fase H (71 righe):

Ciclo 2 - Fase G (40 righe):

Ciclo 2 - Fase H (61 righe):

Ciclo 3 - Fase G (52 righe):

Ciclo 3 - Fase H (43 righe):

Ciclo 4 - Fase G (62 righe):

Ciclo 4 - Fase H (30 righe):

Ciclo 5 - Fase G (35 righe):

Ciclo 5 - Fase H (55 righe):

Ciclo 6 - Fase G (41 righe):

Ciclo 6 - Fase H (62 righe):

Totale righe calcolate: 582


In [15]:
# Cella 10: Visualizzazione risultati per verifica

print("RIEPILOGO CALCOLI COMPLETI")
print("=" * 80)

# Mostriamo un riepilogo per ciclo
for ciclo in range(1, 7):
    calcoli_ciclo = [c for c in calcoli_completi if c['ciclo'] == ciclo]
    
    print(f"\nCICLO {ciclo}:")
    print(f"Numero totale righe: {len(calcoli_ciclo)}")
    
    # Primi 3 risultati fase G
    fase_G = [c for c in calcoli_ciclo if c['fase'] == 'G'][:3]
    print(f"  Fase G (primi 3):")
    for calc in fase_G:
        print(f"    A={calc['A']}, D={calc['D']:.4f}, E={calc['E']:.4f}, F={calc['F']:.4f}")
    
    # Primi 3 risultati fase H
    fase_H = [c for c in calcoli_ciclo if c['fase'] == 'H'][:3]
    print(f"  Fase H (primi 3):")
    for calc in fase_H:
        print(f"    A={calc['A']}, D={calc['D']:.4f}, E={calc['E']:.4f}, F={calc['F']:.4f}")
    
    # Ultimi 3 risultati fase H
    fase_H_ultimi = [c for c in calcoli_ciclo if c['fase'] == 'H'][-3:]
    if fase_H_ultimi:
        print(f"  Fase H (ultimi 3):")
        for calc in fase_H_ultimi:
            print(f"    A={calc['A']}, D={calc['D']:.4f}, E={calc['E']:.4f}, F={calc['F']:.4f}")
            

RIEPILOGO CALCOLI COMPLETI

CICLO 1:
Numero totale righe: 101
  Fase G (primi 3):
    A=1, D=0.0323, E=0.9677, F=-0.9677
    A=2, D=0.0645, E=0.9355, F=-0.9355
    A=3, D=0.0968, E=0.9032, F=-0.9032
  Fase H (primi 3):
    A=71, D=1.0000, E=0.0000, F=0.0000
    A=70, D=0.9859, E=0.0141, F=0.0141
    A=69, D=0.9718, E=0.0282, F=0.0282
  Fase H (ultimi 3):
    A=3, D=0.0423, E=0.9577, F=0.9577
    A=2, D=0.0282, E=0.9718, F=0.9718
    A=1, D=0.0141, E=0.9859, F=0.9859

CICLO 2:
Numero totale righe: 101
  Fase G (primi 3):
    A=1, D=0.0244, E=0.9756, F=-0.9756
    A=2, D=0.0488, E=0.9512, F=-0.9512
    A=3, D=0.0732, E=0.9268, F=-0.9268
  Fase H (primi 3):
    A=61, D=1.0000, E=0.0000, F=0.0000
    A=60, D=0.9836, E=0.0164, F=0.0164
    A=59, D=0.9672, E=0.0328, F=0.0328
  Fase H (ultimi 3):
    A=3, D=0.0492, E=0.9508, F=0.9508
    A=2, D=0.0328, E=0.9672, F=0.9672
    A=1, D=0.0164, E=0.9836, F=0.9836

CICLO 3:
Numero totale righe: 95
  Fase G (primi 3):
    A=1, D=0.0189, E=0.9811, F=

In [16]:
# Cella 11: Confronto con alcuni valori specifici dal file Excel
# Dalla tabella fornita, prendiamo alcuni esempi per verifica

print("CONFRONTO CON VALORI DAL FILE EXCEL")
print("=" * 80)

# Esempio 1: Riga 1 del file (prima riga della tabella)
# D2: =A2/('DATI CICLI'!G$2+1) dove A2=1, G$2=30 (da calcoli: E2-D2 = 45-15 = 30)
# Quindi: D = 1/(30+1) = 1/31 ≈ 0.032258
# E = 1 - D ≈ 0.967742
# F = -E ≈ -0.967742

calc_riga_1 = calcoli_completi[0]  # Prima riga (ciclo 1, fase G, A=1)
print(f"\nRiga 1 (Ciclo 1, Fase G, A=1):")
print(f"  D calcolato: {calc_riga_1['D']:.6f} (atteso: {1/31:.6f})")
print(f"  E calcolato: {calc_riga_1['E']:.6f} (atteso: {1-1/31:.6f})")
print(f"  F calcolato: {calc_riga_1['F']:.6f} (atteso: {-(1-1/31):.6f})")

# Esempio 2: Una riga della fase H del ciclo 1
# Dal file: riga con A=71 (prima riga fase H)
# D: =A/'DATI CICLI'!H$2 dove H$2=71 (da calcoli: F2-E2 = 116-45 = 71)
# Quindi: D = 71/71 = 1.0
# E = 1 - 1 = 0.0
# F = +E = 0.0

# Cerchiamo la riga corrispondente
for calc in calcoli_completi:
    if calc['ciclo'] == 1 and calc['fase'] == 'H' and calc['A'] == 71:
        print(f"\nRiga con A=71 (Ciclo 1, Fase H):")
        print(f"  D calcolato: {calc['D']:.6f} (atteso: 1.000000)")
        print(f"  E calcolato: {calc['E']:.6f} (atteso: 0.000000)")
        print(f"  F calcolato: {calc['F']:.6f} (atteso: 0.000000)")
        break

# Esempio 3: Ultima riga fase H ciclo 1 (A=1)
for calc in calcoli_completi:
    if calc['ciclo'] == 1 and calc['fase'] == 'H' and calc['A'] == 1:
        print(f"\nUltima riga Fase H ciclo 1 (A=1):")
        print(f"  D calcolato: {calc['D']:.6f} (atteso: {1/71:.6f})")
        print(f"  E calcolato: {calc['E']:.6f} (atteso: {1-1/71:.6f})")
        print(f"  F calcolato: {calc['F']:.6f} (atteso: {1-1/71:.6f})")
        break

CONFRONTO CON VALORI DAL FILE EXCEL

Riga 1 (Ciclo 1, Fase G, A=1):
  D calcolato: 0.032258 (atteso: 0.032258)
  E calcolato: 0.967742 (atteso: 0.967742)
  F calcolato: -0.967742 (atteso: -0.967742)

Riga con A=71 (Ciclo 1, Fase H):
  D calcolato: 1.000000 (atteso: 1.000000)
  E calcolato: 0.000000 (atteso: 0.000000)
  F calcolato: 0.000000 (atteso: 0.000000)

Ultima riga Fase H ciclo 1 (A=1):
  D calcolato: 0.014085 (atteso: 0.014085)
  E calcolato: 0.985915 (atteso: 0.985915)
  F calcolato: 0.985915 (atteso: 0.985915)


In [17]:
# Cella 12: Creazione di array numpy con i risultati finali

# Convertiamo i risultati in array numpy per analisi
num_righe = len(calcoli_completi)

# Creiamo array per ogni colonna
col_A = np.zeros(num_righe)
col_D = np.zeros(num_righe)
col_E = np.zeros(num_righe)
col_F = np.zeros(num_righe)
col_ciclo = np.zeros(num_righe, dtype=int)
col_fase = np.empty(num_righe, dtype=object)

for i, calc in enumerate(calcoli_completi):
    col_A[i] = calc['A']
    col_D[i] = calc['D']
    col_E[i] = calc['E']
    col_F[i] = calc['F']
    col_ciclo[i] = calc['ciclo']
    col_fase[i] = calc['fase']

# Statistiche sui risultati
print("STATISTICHE RISULTATI")
print("=" * 80)
print(f"Numero totale di righe: {num_righe}")
print(f"\nValori colonna A:")
print(f"  Minimo: {col_A.min():.0f}, Massimo: {col_A.max():.0f}")
print(f"\nValori colonna D:")
print(f"  Minimo: {col_D.min():.6f}, Massimo: {col_D.max():.6f}, Media: {col_D.mean():.6f}")
print(f"\nValori colonna E:")
print(f"  Minimo: {col_E.min():.6f}, Massimo: {col_E.max():.6f}, Media: {col_E.mean():.6f}")
print(f"\nValori colonna F:")
print(f"  Minimo: {col_F.min():.6f}, Massimo: {col_F.max():.6f}, Media: {col_F.mean():.6f}")

# Conta per ciclo
print(f"\nNumero righe per ciclo:")
for ciclo in range(1, 7):
    count = np.sum(col_ciclo == ciclo)
    print(f"  Ciclo {ciclo}: {count} righe")

STATISTICHE RISULTATI
Numero totale di righe: 582

Valori colonna A:
  Minimo: 1, Massimo: 71

Valori colonna D:
  Minimo: 0.014085, Massimo: 1.000000, Media: 0.505155

Valori colonna E:
  Minimo: 0.000000, Massimo: 0.985915, Media: 0.494845

Valori colonna F:
  Minimo: -0.984127, Massimo: 0.985915, Media: 0.048110

Numero righe per ciclo:
  Ciclo 1: 101 righe
  Ciclo 2: 101 righe
  Ciclo 3: 95 righe
  Ciclo 4: 92 righe
  Ciclo 5: 90 righe
  Ciclo 6: 103 righe


In [18]:
# Cella 13: Funzioni di utilità per lavorare con i dati

def get_calcoli_per_ciclo(numero_ciclo):
    """Restituisce tutti i calcoli per un ciclo specifico"""
    return [calc for calc in calcoli_completi if calc['ciclo'] == numero_ciclo]

def get_calcoli_per_fase(fase):
    """Restituisce tutti i calcoli per una fase specifica (G o H)"""
    return [calc for calc in calcoli_completi if calc['fase'] == fase]

def stampa_sommario_ciclo(numero_ciclo):
    """Stampa un sommario dettagliato per un ciclo"""
    calcoli = get_calcoli_per_ciclo(numero_ciclo)
    fase_G = [c for c in calcoli if c['fase'] == 'G']
    fase_H = [c for c in calcoli if c['fase'] == 'H']
    
    print(f"\nSOMMARIO CICLO {numero_ciclo}")
    print("-" * 50)
    print(f"Righe totali: {len(calcoli)}")
    print(f"Righe fase G: {len(fase_G)}")
    print(f"Righe fase H: {len(fase_H)}")
    
    if fase_G:
        print(f"\nFase G - Primo valore:")
        print(f"  A={fase_G[0]['A']}, D={fase_G[0]['D']:.6f}, E={fase_G[0]['E']:.6f}, F={fase_G[0]['F']:.6f}")
        print(f"\nFase G - Ultimo valore:")
        print(f"  A={fase_G[-1]['A']}, D={fase_G[-1]['D']:.6f}, E={fase_G[-1]['E']:.6f}, F={fase_G[-1]['F']:.6f}")
    
    if fase_H:
        print(f"\nFase H - Primo valore (A più alto):")
        print(f"  A={fase_H[0]['A']}, D={fase_H[0]['D']:.6f}, E={fase_H[0]['E']:.6f}, F={fase_H[0]['F']:.6f}")
        print(f"\nFase H - Ultimo valore (A=1):")
        print(f"  A={fase_H[-1]['A']}, D={fase_H[-1]['D']:.6f}, E={fase_H[-1]['E']:.6f}, F={fase_H[-1]['F']:.6f}")

# Esempio di utilizzo
for ciclo in range(1, 4):  # Mostra solo primi 3 cicli per brevità
    stampa_sommario_ciclo(ciclo)


SOMMARIO CICLO 1
--------------------------------------------------
Righe totali: 101
Righe fase G: 30
Righe fase H: 71

Fase G - Primo valore:
  A=1, D=0.032258, E=0.967742, F=-0.967742

Fase G - Ultimo valore:
  A=30, D=0.967742, E=0.032258, F=-0.032258

Fase H - Primo valore (A più alto):
  A=71, D=1.000000, E=0.000000, F=0.000000

Fase H - Ultimo valore (A=1):
  A=1, D=0.014085, E=0.985915, F=0.985915

SOMMARIO CICLO 2
--------------------------------------------------
Righe totali: 101
Righe fase G: 40
Righe fase H: 61

Fase G - Primo valore:
  A=1, D=0.024390, E=0.975610, F=-0.975610

Fase G - Ultimo valore:
  A=40, D=0.975610, E=0.024390, F=-0.024390

Fase H - Primo valore (A più alto):
  A=61, D=1.000000, E=0.000000, F=0.000000

Fase H - Ultimo valore (A=1):
  A=1, D=0.016393, E=0.983607, F=0.983607

SOMMARIO CICLO 3
--------------------------------------------------
Righe totali: 95
Righe fase G: 52
Righe fase H: 43

Fase G - Primo valore:
  A=1, D=0.018868, E=0.981132, F=-0.